In [2]:
import requests, csv
from bs4 import BeautifulSoup
import re
import pandas as pd
import tqdm
import numpy as np
from collections import OrderedDict

In [1]:
def movie_imdb(imdb_id):
    
    #SELECT SYNOPSIS, GENRES, RATING, NUMBER RATE, YEAR, TITLE, ORIGINAL TITLE, COUNTRY,
    #SELECT LANGUAGE, BUDGET
    
    r1 = requests.get("https://www.imdb.com/title/"+imdb_id)
    bs = BeautifulSoup(r1.text)
    
    # Dictionnary for title, orginal_title, rating, num_rate
    
    info_dict = bs.find('script', {'type': 'application/ld+json'})
    info_dict = info_dict.text
    info_dict = OrderedDict(eval(info_dict))
    
    original_title = info_dict["name"]
    
    if original_title.find("&apos;") != -1:
        original_title = original_title.replace("&apos;", "'")
        
    if original_title.find("&amp") != -1:
        original_title = original_title.replace("&amp", "&")
    
    try :
        title = info_dict["alternateName"]
    except KeyError:
        title = info_dict["name"]
    
    if title.find("&amp") != -1:
        title = title.replace("&amp", "&")
        
    if title.find("&apos;") != -1:
        title = title.replace("&apos;", "'")
    
    try :      
        rating = info_dict['aggregateRating']['ratingValue']
        num_rate = info_dict['aggregateRating']['ratingCount']
    except KeyError:
        rating = ""
        num_rate = ""
    
    #Dictionnary for budget, bud_currency
    
    info_dict_2 = bs.findAll('script', {'id': re.compile('__NEXT_DATA__')})
    info_dict_2 = [text.text for text in info_dict_2][0]
    info_dict_2 = OrderedDict(eval(info_dict_2, {'null': None, 'false':None, 'true':None}))
    info_dict_2_1 = info_dict_2['props']['urqlState']
    
    for key in info_dict_2_1.keys():
        try:
            budget = info_dict_2_1[key]['data']['title']['productionBudget']
            def_key = key
        except KeyError:
            continue
    
    if budget == None:
        budget = ""
        bud_currency = ""
    else : 
        budget = info_dict_2_1[def_key]['data']['title']['productionBudget']["budget"]["amount"]
        bud_currency = info_dict_2_1[def_key]['data']['title']['productionBudget']["budget"]["currency"]
    
    
    year = bs.findAll('a', {'class': re.compile('ipc-link')})
    year = int([text.text for text in year][0].strip("– "))
    
    genre_bs = bs.findAll('a', {'class': re.compile('ipc-metadata-list-item')})
    genre_bs = [text.text for text in genre_bs]
    
    genre_list = ["Action", "Adult", "Adventure", "Animation", "Biography", "Comedy", "Crime", "Documentary", 
              "Drama", "Family", "Fantasy", "Film-Noir", "History", "Horror", "Music", "Musical", "Mystery", 
              "News", "Reality-TV", "Romance", "Sci-Fi", "Short", "Sport", "Thriller", "War", "Western"]
    
    genres = [genre for genre in genre_bs if genre in genre_list]
    
    try :
        syn = bs.findAll('div', {'class': 'ipc-html-content ipc-html-content--base'})[0]
        syn = [text.text for text in syn]
        synopsis = syn[0].split(" —")[0]
    except IndexError:
        synopsis = ""
    
    
    countries = bs.findAll('a', {'href': re.compile('country_of_origin')})
    if len(countries) == 0:
        country = ['None']
    else:
        country = ["USA" if text.text == "United States" else "UK" if text.text == "United Kingdom" \
                    else text.text for text in countries]
    
    
    
    language = bs.findAll('a', {'href': re.compile('primary_language')})
    if len(language) == 0:
        language = ["No Info"]
    else : 
        language = [text.text for text in language]
    
      
    #SELECT CASTING, DIRECTOR, WRITERS
    
    r2 = requests.get("https://www.imdb.com/title/"+imdb_id+"/fullcredits")
    bs2 = BeautifulSoup(r2.text)

    #select casting
    cast_list = bs2.find_all('table', {'class': 'cast_list'})
    if cast_list == None or cast_list == []:
        cast = [""]
        cast_code = [""]
    else :
        cast_list_test = bs2.find('table', {'class': 'cast_list'}).find_all('td', {'class':'primary_photo'})
        if len(cast_list_test) >= 30 : 
            cast = [str(cast_list).split('img alt="')[i].split('" class="')[0] for i in range(1,31)]
            cast_code = [str(cast_list_test).split('/name/')[i].split('/">')[0] for i in range(1,31)]
        else : 
            cast = [str(cast_list).split('img alt="')[i].split('" class="')[0] \
                    for i in range(1,len(cast_list_test)+1)]
            cast_code = [str(cast_list_test).split('/name/')[i].split('/">')[0] \
                          for i in range(1,len(cast_list_test)+1)]
    
    #select director
    test_dir = bs2.find('h4', {'id':'director'})
    if test_dir == None:
        director = [""]
        dir_code = [""]
    else : 
        direct = bs2.find('table', {'class': 'simpleTable simpleCreditsTable'}) \
                .find_all('td', {'class': 'name'})
        direct_t = [text.text for text in direct]
        director = [i.strip() for i in direct_t]
        dir_code = [(str(direct).split('href="/name/')[i+1]).split('/"> ')[0] for i in range(len(director))]
    
    #SELECT RUNTIME
    
    r3 = requests.get("https://www.imdb.com/title/"+imdb_id+"/technical")
    bs3 = BeautifulSoup(r3.text)

    time = bs3.find('table', {'class': 'dataTable labelValueTable'}).find_all('tr', {'class': 'odd'})
    runtime = [text.text for text in time]
    try:
        if ("min (" in (runtime[0].strip())) and (" min)" in (runtime[0].strip())):
            duration = int((runtime[0].strip().split("min (")[1]).split(" min)")[0])
        elif "hr (" in (runtime[0].strip()): 
            duration = int((runtime[0].strip().split("hr (")[1]).split(" min)")[0])
        else :
            duration = int((runtime[0].strip().split("Runtime")[1]).split(" min")[0])
    except (ValueError, IndexError):
        duration = ""
        
        
    #SELECT PRODUCTION
    
    r4 = requests.get("https://www.imdb.com/title/"+imdb_id+"/companycredits")
    bs4 = BeautifulSoup(r4.content)
    test = bs4.find('h4', {'id': 'production'})
    if test == None:
        production = [""]
        prod_code = [""]
    else : 
        prod_list = bs4.find('ul', {'class': 'simpleList'}).find_all("li")
        prod_t = [text.text.strip() for text in prod_list]
        production = [pro.split("   ")[0] if "   " in pro else pro for pro in prod_t]
        
        prod_code = [(str(i).split("pany/")[1]).split('">')[0] for i in prod_list]
        
    if rating != "":
        return imdb_id, title, original_title, year, director, dir_code, cast, cast_code, genres, duration, \
        country, language, production, prod_code, synopsis, rating, num_rate, budget, bud_currency
    else:
        return imdb_id,"","","","","","","","","","","","","","","","","",""

In [3]:
df = pd.read_excel('movies_to_scrap/to_scrap.xlsx')

In [3]:
df = pd.read_csv('movies_to_scrap/scrap_it.csv')

In [4]:
imdb_id = df["imdb"].values

In [5]:
db_df = pd.read_csv('new_sql/movies_sql.csv')

In [6]:
imdb_db = db_df["imdb_id"].values

In [ ]:
final_imdb = [i for i in imdb_id if i not in imdb_db]

In [10]:
movies = []

In [12]:
for movie in tqdm.tqdm(final_imdb):
    movies.append(movie_imdb(movie))

100%|██████████| 17/17 [01:05<00:00,  3.87s/it]


In [ ]:
movie_df = pd.DataFrame(movies)

In [329]:
movie_keep = movie_df.drop(movie_df[movie_df[3] == ""].index)
too_soon = pd.DataFrame(movie_df[0].drop(movie_df[movie_df[3] != ""].index).values, columns = ['imdb_id'])

In [ ]:
too_soon.to_csv('movies_to_scrap/too_soon.csv')

## Lists of data

In [20]:
imdb = list(movie_keep[0].values)
title = list(movie_keep[1].values)
original_title = list(movie_keep[2].values)
year = list(movie_keep[3].values)
director = list(movie_keep[4].values)
dir_code = list(movie_keep[5].values)
cast = list(movie_keep[6].values)
cast_code = list(movie_keep[7].values)
genres = list(movie_keep[8].values)
duration = list(movie_keep[9].values)
country = list(movie_keep[10].values)
language = list(movie_keep[11].values)
production = list(movie_keep[12].values)
prod_code = list(movie_keep[13].values)
synopsis = list(movie_keep[14].values)
rating = list(movie_keep[15].values)
num_rate = list(movie_keep[16].values)
budget = list(movie_keep[17].values)
bud_currency = list(movie_keep[18].values)

In [21]:
saw = ["" for i in range(len(imdb))]

## Update country_sql

In [22]:
state_df = pd.read_csv('new_sql/country_sql.csv')

In [23]:
code_state = list(state_df.id.values)
countries = list(state_df.country.values)

In [24]:
max_country_id = max(code_state)

In [25]:
update_state = [country[i][j] for i in range(len(country)) for j in range(len(country[i])) \
                if country[i][j] not in countries]

In [26]:
update_state_code = []

In [27]:
for i in range(len(country)): 
    for j in range(len(country[i])):
        if country[i][j] not in countries:
            update_state_code.append(max_country_id + 1)
            max_country_id += 1

In [28]:
state_data = {"id": update_state_code, "country": update_state}

In [29]:
up_country = pd.DataFrame(state_data).set_index("id")

In [42]:
up_country.to_csv("update_sql/up_country.csv")

## Update language_sql

In [31]:
lan_df = pd.read_csv('new_sql/language_sql.csv')

In [32]:
code_lan = list(lan_df.id.values)
lan = list(lan_df.language.values)

In [33]:
max_language_id = max(code_lan)

In [34]:
update_lan = [language[i][j] for i in range(len(language)) for j in range(len(language[i])) \
                if language[i][j] not in lan and language[i][j] != ""]

In [35]:
update_lan_code = []

In [36]:
for i in range(len(language)): 
    for j in range(len(language[i])):
        if language[i][j] not in lan and language[i][j] != "":
            update_lan_code.append(max_language_id + 1)
            max_language_id += 1

In [38]:
lan_data = {"id": update_lan_code, "language": update_lan}

In [39]:
up_language = pd.DataFrame(lan_data).set_index("id")

In [40]:
up_language.to_csv("update_sql/up_language.csv")

## Creation of genre_code, country_code and language_code for data

In [43]:
genre_df = pd.read_csv("new_sql/genre_sql.csv")
country_df = pd.read_csv("new_sql/country_sql.csv")
language_df = pd.read_csv("new_sql/language_sql.csv")

country_df2 = pd.read_csv("update_sql/up_country.csv")
language_df2 = pd.read_csv("update_sql/up_language.csv")

In [44]:
country_df = pd.concat([country_df, country_df2])

In [45]:
language_df = pd.concat([language_df, language_df2])

In [46]:
genre_code = [genre_df.id.values[list(genre_df.genre.values).index(genres[i][j])] \
             for i in range(len(genres)) for j in range(len(genres[i]))]

country_code = [country_df.id.values[list(country_df.country.values).index(country[i][j])] \
                for i in range(len(country)) for j in range(len(country[i]))]

language_code = [language_df.id.values[list(language_df.language.values).index(language[i][j])] \
                 for i in range(len(language)) for j in range(len(language[i]))]

## Make change of currencies

In [47]:
changer = pd.read_excel("currency_change.xlsx")

In [48]:
def_budget = []

In [49]:
for i in range(len(bud_currency)):
    if bud_currency[i] == "":
        def_budget.append("")
    else:
        def_budget.append(int(budget[i] * changer["Change"].values[list(changer.Currencies.values).index(bud_currency[i])]))

## Update selection_film

#### List to string for directors, actors, genres, countries, language, production

In [50]:
dict_dir = {}
dict_cast = {}
dict_genre = {}
dict_country = {}
dict_language = {}
dict_prod = {}

In [51]:
temp_dir = []
temp_cast = []
temp_genre = []
temp_country = []
temp_language = []
temp_prod = []

In [52]:
ids = imdb[0]

In [53]:
for i in range(len(imdb)):
    for j in range(len(director[i])):
        if ids == imdb[i]:
            temp_dir.append(director[i][j]) 
        elif ids != imdb[i]:
            dict_dir[ids] = " | ".join(temp_dir)
            ids = imdb[i]
            temp_dir = []
            temp_dir.append(director[i][j])
    dict_dir[ids] = " | ".join(temp_dir)

In [54]:
ids = imdb[0]

In [55]:
for i in range(len(imdb)):
    for j in range(len(cast[i])):
        if ids == imdb[i]:
            temp_cast.append(cast[i][j])
        elif ids != imdb[i]:
            dict_cast[ids] = " | ".join(temp_cast)
            ids = imdb[i]
            temp_cast = []
            temp_cast.append(cast[i][j])
    dict_cast[ids] = " | ".join(temp_cast)

In [117]:
ids = imdb[0]

In [56]:
for i in range(len(imdb)):
    for j in range(len(genres[i])):
        if ids == imdb[i]:
            temp_genre.append(genres[i][j])
        elif ids != imdb[i]:
            dict_genre[ids] = " | ".join(temp_genre)
            ids = imdb[i]
            temp_genre = []
            temp_genre.append(genres[i][j])
    dict_genre[ids] = " | ".join(temp_genre)

In [57]:
ids = imdb[0]

In [58]:
for i in range(len(imdb)):
    for j in range(len(country[i])):
        if ids == imdb[i]:
            temp_country.append(country[i][j])
        elif ids != imdb[i]:
            dict_country[ids] = " | ".join(temp_country)
            ids = imdb[i]
            temp_country = []
            temp_country.append(country[i][j])
    dict_country[ids] = " | ".join(temp_country)

In [59]:
ids = imdb[0]

In [60]:
for i in range(len(imdb)):
    for j in range(len(language[i])):
        if ids == imdb[i]:
            temp_language.append(language[i][j])
        elif ids != imdb[i]:
            dict_language[ids] = " | ".join(temp_language)
            ids = imdb[i]
            temp_language = []
            temp_language.append(language[i][j])
    dict_language[ids] = " | ".join(temp_language)

In [61]:
ids = imdb[0]

In [62]:
for i in range(len(imdb)):
    for j in range(len(production[i])):
        if ids == imdb[i]:
            temp_prod.append(production[i][j])
        elif ids != imdb[i]:
            dict_prod[ids] = " | ".join(temp_prod)
            ids = imdb[i]
            temp_prod = []
            temp_prod.append(production[i][j])
    dict_prod[ids] = " | ".join(temp_prod)

#### Put data in a dataframe

In [63]:
select_data = {"imdb_id": imdb, "vu": saw, "à voir": saw, "title": title, "original_title": original_title, \
               "year": year, "director": list(dict_dir.values()), "casting": list(dict_cast.values()), \
               "genres": list(dict_genre.values()), "duration": duration, \
               "country": list(dict_country.values()), "language": list(dict_language.values()), \
               "production": list(dict_prod.values()), "synopsis": synopsis, "rating": rating, \
               "num_rate": num_rate, "budget": def_budget}

In [64]:
select = pd.DataFrame(select_data).set_index("imdb_id")

In [65]:
select.to_csv("movies_scraped.csv")

## Update movie_sql

In [66]:
movie_data = {"imdb_id": imdb, "title": title, "original_title": original_title, "year": year, \
               "duration": duration, "rating": rating, "num_rate": num_rate, "budget": def_budget, \
               "synopsis": synopsis}

In [67]:
up_movie = pd.DataFrame(movie_data).set_index("imdb_id")

In [68]:
up_movie.to_csv("update_sql/up_movie.csv")

## Update actor_sql

In [69]:
actor_df = pd.read_csv('new_sql/actor_sql.csv')

In [70]:
code_actor = list(actor_df.code.values)

In [71]:
update_cast_code = []
update_cast = []

In [72]:
for i in range(len(cast_code)): 
    for j in range(len(cast_code[i])):
        if cast_code[i][j] not in code_actor:
            update_cast_code.append(cast_code[i][j])
            update_cast.append(cast[i][j])
            code_actor.append(cast_code[i][j])

In [73]:
actor_data = {"code": update_cast_code, "casting": update_cast}

In [74]:
up_actor = pd.DataFrame(actor_data).set_index("code")

In [75]:
if len(update_cast) > 0:
    up_actor.to_csv("update_sql/up_actor.csv")

## Update director_sql

In [76]:
dir_df = pd.read_csv('new_sql/director_sql.csv')

In [77]:
code_dir = list(dir_df.code_name.values)

In [78]:
update_dir_code = []
update_dir = []

In [79]:
for i in range(len(dir_code)): 
    for j in range(len(dir_code[i])):
        if dir_code[i][j] not in code_dir:
            update_dir_code.append(dir_code[i][j])
            update_dir.append(director[i][j])
            code_dir.append(dir_code[i][j])

In [80]:
dir_data = {"code": update_dir_code, "director": update_dir}

In [81]:
up_dir = pd.DataFrame(dir_data).set_index("code")

In [82]:
if len(update_dir) > 0:
    up_dir.to_csv("update_sql/up_dir.csv")

## Update production_sql

In [83]:
prod_df = pd.read_csv('new_sql/production_sql.csv')

In [84]:
code_prod = list(prod_df.prod_code.values)

In [85]:
update_prod_code = []
update_prod = []

In [86]:
for i in range(len(prod_code)): 
    for j in range(len(prod_code[i])):
        if prod_code[i][j] not in code_prod:
            update_prod_code.append(prod_code[i][j])
            update_prod.append(production[i][j])
            code_prod.append(prod_code[i][j])

In [87]:
prod_data = {"code": update_prod_code, "production": update_prod}

In [88]:
up_prod = pd.DataFrame(prod_data).set_index("code")

In [89]:
if len(update_prod) > 0:
    up_prod.to_csv("update_sql/up_prod.csv")

## Update movie_dir_sql

In [90]:
dir_imdb = [imdb[i] for i in range(len(imdb)) for j in range(len(dir_code[i]))]
all_dir = [dir_code[i][j] for i in range(len(imdb)) for j in range(len(dir_code[i]))]

In [91]:
movie_dir_data = {"imdb_id": dir_imdb, "code": all_dir}

In [92]:
up_movie_dir = pd.DataFrame(movie_dir_data)

In [ ]:
up_movie_dir = up_movie_dir.drop(up_movie_dir[up_movie_dir["code"] == ""].index).set_index("imdb_id")

In [93]:
up_movie_dir.to_csv("update_sql/up_movie_dir.csv")

## Update movie_actor_sql

In [94]:
cast_imdb = [imdb[i] for i in range(len(imdb)) for j in range(len(cast_code[i]))]
all_actor = [cast_code[i][j] for i in range(len(imdb)) for j in range(len(cast_code[i]))]

In [95]:
movie_cast_data = {"imdb_id": cast_imdb, "code": all_actor}

In [96]:
up_movie_cast = pd.DataFrame(movie_cast_data)

In [ ]:
up_movie_cast = up_movie_cast.drop(up_movie_cast[up_movie_cast["code"] == ""].index).set_index("imdb_id")

In [97]:
up_movie_cast.to_csv("update_sql/up_movie_cast.csv")

## Update movie_prod_sql

In [98]:
prod_imdb = [imdb[i] for i in range(len(imdb)) for j in range(len(prod_code[i]))]
all_prod = [prod_code[i][j] for i in range(len(imdb)) for j in range(len(prod_code[i]))]

In [99]:
movie_prod_data = {"imdb_id": prod_imdb, "code": all_prod}

In [100]:
up_movie_prod = pd.DataFrame(movie_prod_data)

In [ ]:
up_movie_prod = up_movie_prod.drop(up_movie_prod[up_movie_prod["code"] == ""].index).set_index("imdb_id")

In [101]:
up_movie_prod.to_csv("update_sql/up_movie_prod.csv")

## Update movie_genre_sql

In [102]:
genre_imdb = [imdb[i] for i in range(len(imdb)) for j in range(len(genres[i]))]

In [103]:
movie_genre_data = {"imdb_id": genre_imdb, "code": genre_code}

In [104]:
up_movie_genre = pd.DataFrame(movie_genre_data)

In [ ]:
up_movie_genre = up_movie_genre.drop(up_movie_genre[up_movie_genre["code"] == ""].index).set_index("imdb_id")

In [105]:
up_movie_genre.to_csv("update_sql/up_movie_genre.csv")

## Update movie_country_sql

In [106]:
country_imdb = [imdb[i] for i in range(len(imdb)) for j in range(len(country[i]))]

In [107]:
movie_country_data = {"imdb_id": country_imdb, "code": country_code}

In [108]:
up_movie_country = pd.DataFrame(movie_country_data)

In [ ]:
up_movie_country = up_movie_country.drop(up_movie_country[up_movie_country["code"] == ""].index).set_index("imdb_id")

In [109]:
up_movie_country.to_csv("update_sql/up_movie_country.csv")

## Update movie_language_sql

In [110]:
language_imdb = [imdb[i] for i in range(len(imdb)) for j in range(len(language[i]))]

In [111]:
movie_language_data = {"imdb_id": language_imdb, "code": language_code}

In [112]:
up_movie_language = pd.DataFrame(movie_language_data)

In [ ]:
up_movie_language = up_movie_language.drop(up_movie_language[up_movie_language["code"] == ""].index).set_index("imdb_id")

In [113]:
up_movie_language.to_csv("update_sql/up_movie_language.csv")